In [ ]:
from google.cloud import bigquery

In [ ]:
#Creat a 'Client'object
client = bigquert.Client()
#construct a reference to the "openaq" dataset
dataset_ref = client.dataset('openaq', project = 'bigquery-public-data')
#API request-fetch the dataset
dataset = client.get_dataset(dataset_ref)
#list all the tables in the "openaq" dataset
tables = list(client.list_tables(dataset))
#Print names of all tables in the dataset(there's only one!)
for table in tables:
    print(table.table_id)


In [ ]:
Using Kaggle's public dataset BigQuery integration.
global_air_quality

In [ ]:
The dataset contains only one table, called global_air_quality.
We'll fetch the table and take a peek at the first few rows to see what sort of data it contains.
(Again, we have hidden the code. To take a peek, click on the "Code" button below.)

In [ ]:
# Construct a reference to the "global_air_quality" table
table_ref = dataset_ref.table("global_air_quality")
# API request - fetch the table
table = client.get_table(table_ref)
# Preview the first five lines of the "global_air_quality" table
client.list_rows(table, max_results= 5).to_dataframe()

In [ ]:
#query to select all the items from the "city" column 
#where the "country" column is "US"
query = """
        SELECT city
        FROM 'bigquery-public-data.openaq.global_air_quality'
        WHERE country = 'US'
        """

### Submitting the query to the dataset

In [ ]:
#Create a "Client" object
client = bigquery.Client()

In [ ]:
#Set up the query
query_job = client.query(query)

In [ ]:
Next, we run the query and convert the results to a pandas DataFrame.

In [ ]:
# API request - run the query, and return a pandas Dataframe
us_cities = query_job.to_dataframe()

In [ ]:
Now we've got a pandas DataFrame called us_cities, 
which we can use like any other DataFrame.

In [ ]:
# What five cities have the most measurements?

In [ ]:
us_cities.city.value_counts().head()

In [ ]:
#Query to get the score column from every row where the type column has value "job"
query = """
        SELECT score, title
        FROM 'bigquery-public-data.hacker_news.full'
        WHERE type = 'job'
        """


In [ ]:
#Create a QueryJobConfig object to estimate size of query without running it
dry_run_config = bigquery.QueryJobConfig(dry_run = True)


In [ ]:
#API request- dry run query to estimate costs
dry_run_query_job = client.query(query, job_config= dry_run_config)


In [ ]:
print('This query will process {} bytes.'.format(dry_run_query_job.total_bytes_processed))


In [ ]:
This query will process 399514186 bytes.

In [ ]:
You can also specify a parameter when running the query to limit how much data you are willing to scan. Here's an example with a low limit.

In [ ]:
#Only run the query if it's less than 1 MB
ONE_MB = 1000*1000
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = ONE_MB)


In [ ]:
#Set up the query(will only run if it's less than 1 MB)
safe_query_job = client.query(query, job_config = safe_config)
safe_query_job.to_dataframe()

In [ ]:
In this case, the query was cancelled, because the limit of 1 MB was exceeded. However, we can increase the limit to run the query successfully!

In [ ]:
# Only run the query if it's less than 1 GB
ONR_GB = 1000*1000*1000
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = ONE_GB)


In [ ]:
#Set up the query(will only run if it's less than 1 GB)
safe_query_job = client.query(query, job_config = safe_config)
#API request - try to run query, and return a pandas DataFrame
job_post_scores = safe_query_job.to_dataframe()
#Print average Score for job posts
job_post_scores.score.mean()

In [ ]:
# Query to select countries with units of "ppm"
!!!
#注意⚠️
#`bigquery-public-data.openaq.global_air_quality`
#上面是不是单引号''，而是``两个单向号（在键盘右上角esc下面👇那个键）

first_query = """
              SELECT country
              FROM `bigquery-public-data.openaq.global_air_quality`
              WHERE unit = "ppm"
              """
# Your code goes here

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 10 GB)

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
first_query_job = client.query(first_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
first_results = first_query_job.to_dataframe()

# View top few rows of results
print(first_results.head())

# Check your answer
q_1.check()

In [ ]:
#Query to select comments that received more than 10 replies
query_popular = """
                SELECT parent, COUNT(id)
                FROM `bigquery-publio-data.hacker_news.comments`
                GROUP BY parent
                HAVING COUNT(id) > 10
                """

In [ ]:
#Now that our query is ready, let's run it and store the results in a pandas DataFrame:

In [ ]:
# Set up the query(cancel the query if it would use too much of
# your quota, with the limit set to 10 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 10**10)
query_job = client.query(query_popular, job_config = safe_config)

# API request - run the query, and convert the results to a pandas DataFrame
popular_comments = query_job.to_dataframe()

#Print the first five rows of the DataFram
popular_comments.head()

In [ ]:
Aliasing and other improvements
as = 'Aliasing'#别名

In [ ]:
# Improved version of earlier query, now with aliasing & improved readability
query_improved = """
                SELECT parent, COUNT(1) AS NumPosts
                FROM `bigquery-publio-data.hacker_news.comments`
                GROUP BY parent
                HAVING COUNT(id) > 10
                """
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query_improved, job_config=safe_config)

# API request - run the query, and convert the results to a pandas DataFrame
improved_df = query_job.to_dataframe()

# Print the first five rows of the DataFrame
improved_df.head()

In [ ]:
#And this query won't work, because the author column isn't passed to an aggregate function or a GROUP BY clause:
query_bad = """
            SELECT author, parent, COUNT(id)
            FROM `bigquery-public-data.hacker_news.comments`
            GROUP BY parent
            """
#⚠️：if make this error, you'll get error message SELLECT list expression references
#column (column's) which is neither grouped nor aggregated at.

In [ ]:
ORDER by ASC/ DESC(short for 'descending')

In [ ]:
DATE and DATETIME

In [ ]:
YYYY-[M]M-[D]D
- YYYY: Four-digit year
- [M]M: One or two digit month
- [D]D: One or two digit day
So 2019-01-10 is interpreted as January 10, 2019.
The DATETIME format is like the dateformat... but with time added at the end.

In [ ]:
# Query to find out the number of accidents for each day of the week
query = """
        SELECT COUNT(consecutive_number) AS num_accidents,
               EXTRACT(DAYOFWEEK FROM timestamp_of_crash) AS day_of_week
        FROM `bigquery-public-data.nhtsa_traffic_fatalities.accident_2015`
        GROUP BY day_of_week
        ORDER BY num_accidents DESC
        """
        

In [ ]:
#Set up the query(cancel the query if it would use too much of
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QuerJobConfig(maximum_bytes_biled= 10**9)
query_job = client.query(query, job_config= safe_config)

# API request - run the query, and convert the results to a pandas DataFrame
accidents_by_day = query_job.to_dataframe()

#print the DataFrame
accidents_by_day


In [ ]:
Requirements:
- Your results should have the country name rather than the country code. You will have one row for each country.
- The aggregate function for average is **AVG()**.  Use the name `avg_ed_spending_pct` for the column created by this aggregation.
- Order the results so the countries that spend the largest fraction of GDP on education show up first.

country_spend_pct_query = """
                          SELECT country_name, AVG(value) AS avg_ed_spending_pct
                          FROM `bigquery-public-data.world_bank_intl_education.international_education`
                          WHERE indicator_code = 'SE.XPD.TOTL.GD.ZS' and year >= 2010 and year <= 2017
                          GROUP BY country_name
                          ORDER BY avg_ed_spending_pct DESC
                          """
Requirements:
- You should have one row for each indicator code.
- The columns in your results should be called `indicator_code`, `indicator_name`, and `num_rows`.
- Only select codes with 175 or more rows in the raw database (exactly 175 rows would be included).
- To get both the `indicator_code` and `indicator_name` in your resulting DataFrame, you need to include both in your **SELECT** statement (in addition to a **COUNT()** aggregation). This requires you to include both in your **GROUP BY** clause.
- Order from results most frequent to least frequent.

code_count_query = """
                   SELECT indicator_code, indicator_name,
                   COUNT(1) AS num_row 
                   FROM `bigquery-public-data.world_bank_intl_education.international_education`
                   WHERE year = 2016
                   GROUP BY indicator_code, indicator_name
                       HAVING COUNT(1) >= 175
                   ORDER BY COUNT(1) DESC
                   """